### David Herrera
### Estid Lozano

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openml as oml

# Exercise 1

**2 pts**

Let $K$ be the Gaussian kernel and let $D = \{(x_1, y_1), ..., (x_n, y_n)\}$ be a dataset. Now consider the empirical reproducing kernel map $φ$ with $φ(x) = K(x, ·)$, and suppose that we use this kernel map to generate new features based on these “landmarks”. Formally, let $D' = \{(φ(x)(D), y) | (x, y) ∈ D\}$ be the dataset we obtain if we have one feature for each datapoint indicating the similarity as per the Gaussian kernel $K$. So $D' ∈ R^{n×n}$.

Show that it is wrong to say that kernels produce new features by showing that the dot product of two arbitrary points $x_i$, $x_j$ in $D'$ is not identical to the kernel value $K(x_i, x_j)$.

**proof**:

...

# Exercise 2

**(Raw Empirical vs. corrected Empirical Kernel Map - 4 pts)**

**2.1.** Implement the polynomial kernal and the Gaussian kernel explicitly.

In [ ]:
# polynomial

In [ ]:
# gaussian

**2.2.** Write a function **mapDataset(X, kernel, corrected=True)** that takes a dataset (only the attribute not the labels) and explicitly computes the empirical feature map of the given kernel (functional argument). Depending on whether **corrected** is true, it should adjust the data using the $K^{−1/2}$ matrix.

In [ ]:
def mapDataset(X, kernel, corrected=True):
    # compute empirical feature map
    
    # corrected
    if corrected:
        # adjust the data

**2.3.** Explicitly test that using the corrected version, the kernel computes the correct dot product in the feature space when applying both polynomial and Gaussian kernel to the iris, amazon, and madelon dataset.

In [ ]:
# datasets
dtsetIri = oml.datasets.get_dataset(61) # iris
XIri, yIri, catIndIri, attrsIri = dtsetIri.get_data()

dtsetAma = oml.datasets.get_dataset(1457) # amazon-commerce-reviews
XAma, yAma, catIndAma, attrsAma = dtsetAma.get_data()

dtsetMad = oml.datasets.get_dataset(1485) # madelon
XMad, yMad, catIndMad, attrsMad = dtsetMad.get_data()

# df = pd.concat([X,pd.DataFrame({"label":y})],axis=1)

In [ ]:
# test

**2.4.** Report performance of Logistic Regression, Naive Bayes and Decision Trees when applied to these three (mapped) datasets using the linear kernel, the quadratic kernel (each of which with $c ∈ \{0, 1, 10\}$) and with the Gaussian kernel (for values of $σ ∈ \{0.5, 1, 10\}$).


In [ ]:
# performance

# Exercise 3

**2 pts**

Create an artificial dataset with two attributes and two classes. There should be 100 instances in the range of 1 around the origin and 400 instances outside the unit sphere (but in a range of less than 10) uniformly distributed. Visualize your data. Now map your data with the feature map corresponding to the quadratic kernel for different values of c into a 3D-space. Create 3D plots (with appropriate axes labels) and explain what you observe and in how far linear separability is changed.

In [ ]:
# artificial dataset


In [ ]:
# visualize

In [ ]:
# map data

In [ ]:
# 3D plots

**Answer:** We observe...